In [ ]:
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from ResNet_Classifier import classifierTrain
from torch.utils.data import WeightedRandomSampler, DataLoader
import numpy as np
import torch

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [ ]:
train_dataset_longtail = datasets.ImageFolder(root='./datasets_longtail/train', transform=transform)
train_loader_longtail = DataLoader(train_dataset_longtail, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)

test_dataset_longtail = datasets.ImageFolder(root='./datasets_longtail/test', transform=transform)
test_loader_longtail = DataLoader(test_dataset_longtail, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
classifierTrain(50,train_loader_longtail,test_loader_longtail,'SVM')

In [ ]:
targets = train_dataset_longtail.targets
class_counts = np.bincount(targets)
num_classes = len(class_counts)

class_weights = 1. / class_counts
sample_weights = [class_weights[t] for t in targets]
sample_weights = torch.DoubleTensor(sample_weights)

sampler = WeightedRandomSampler(weights=sample_weights,
                                num_samples=len(sample_weights),
                                replacement=True)

train_loader_resampling = DataLoader(train_dataset_longtail,
                                   batch_size=32,
                                   sampler=sampler,
                                   num_workers=4,
                                   pin_memory=True)

In [ ]:
classifierTrain(50,train_loader_resampling,test_loader_longtail,'SVM')